<a href="https://colab.research.google.com/github/Pintu1-data/OptimizeCNNModel/blob/main/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Create CNN Model and Optimize it using Keras Tuner* **

In [1]:
!pip install keras.tuner

     |████████████████████████████████| 98 kB 5.6 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.8.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_image,train_labels),(test_image,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_image/255.0
test_image=test_image/255.0

In [8]:
train_image[0].shape

(28, 28)

In [9]:
train_image=train_image.reshape(len(train_image),28,28,1)


In [10]:
test_image=test_image.reshape(len(test_image),28,28,1)

In [16]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [17]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [18]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [19]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 01m 22s]
val_accuracy: 0.8583333492279053

Best val_accuracy So Far: 0.9098333120346069
Total elapsed time: 00h 06m 13s
INFO:tensorflow:Oracle triggered exit


In [20]:
model=tuner_search.get_best_models(num_models=1)[0]

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        76864     
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 96)                2973792   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 3,052,106
Trainable params: 3,052,106
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1219 - accuracy: 0.9551 - val_loss: 0.2654 - val_accuracy: 0.9168
Epoch 5/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0846 - accuracy: 0.9678 - val_loss: 0.3020 - val_accuracy: 0.9165
Epoch 6/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0559 - accuracy: 0.9793 - val_loss: 0.3559 - val_accuracy: 0.9152
Epoch 7/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0414 - accuracy: 0.9851 - val_loss: 0.4164 - val_accuracy: 0.9150
Epoch 8/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0303 - accuracy: 0.9891 - val_loss: 0.4723 - val_accuracy: 0.9157
Epoch 9/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0258 - accuracy: 0.9907 - val_loss: 0.5330 - val_accuracy: 0.9102
Epoch 10/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0241 - accuracy: 0.9919 - val_loss: 0.5709 - val_a